In [ ]:
pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import PKCS1_OAEP, AES # Importing PKCS1_OAEP for RSA encryption/decryption and AES for symmetric key encryption.
from Crypto.Protocol.KDF import PBKDF2 # Importing PBKDF2, a key derivation function, commonly used to derive secure keys
from Crypto.PublicKey import RSA #  Importing RSA for asymmetric key generation, encryption, and decryption functionalities.
from Crypto.Signature import pkcs1_15 # Importing pkcs1_15 for RSA-based signing and signature verification.
from Crypto.Hash import SHA256 # Importing SHA256, a secure hashing algorithm for generating hash values.
from Crypto.Random import get_random_bytes #  Importing a function to generate cryptographically secure random bytes, useful for keys or IVs.
from Crypto.Util.Padding import pad, unpad #  Importing padding utilities to adjust data block size to fit AES requirements.
from Crypto.Hash import SHA256 #(Duplicate) Importing SHA256 again, a hashing algorithm used for cryptographic hash functions.

def generate_rsa_keys():
    """Generate RSA key pairs for digital signature."""
    key = RSA.generate(2048)
    # Generates a new RSA key pair with a strength of 2048 bits, which is considered secure for modern standards.
    private_key = key.export_key()
    #  Extracts and exports the private key from the generated RSA key pair.
    public_key = key.publickey().export_key()
    # Extracts the public key from the generated RSA key pair and exports it.
    return private_key, public_key

def sign_message(message, private_key):
    #  Defines a function to sign a message using an RSA private key.
    key = RSA.import_key(private_key)
    # Imports the RSA private key for use in signing.
    h = SHA256.new(message)
    # Creates a SHA-256 hash object from the provided message. This hash will be signed.
    signature = pkcs1_15.new(key).sign(h)
    # Signs the hash of the message using the imported private key and PKCS#1.5 padding scheme, creating the digital signature.
    return signature
    # Create a SHA-256 hash object from the message
    h = SHA256.new(message.encode())
    # Sign the hash using PKCS#1.5 padding
    signature = pkcs1_15.new(key).sign(h)
    # : (Redundant) Again signs the hash of the message, duplicating the earlier action.
    return signature

def verify_signature(message, signature, public_key):
    # Import the public key
    key = RSA.import_key(public_key)
      # Create a SHA-256 hash object from the message
    h = SHA256.new(message)
    try:
        # Verify the signature using PKCS#1.5 padding
        pkcs1_15.new(key).verify(h, signature)
        return True  # Signature is valid
    except (ValueError, TypeError):
        return False  # Signature is not valid



def encrypt_file(file_path, public_key_path, output_path, private_key_sender):
    # Defines a function for encrypting a file. It takes the path of the file
    # to be encrypted, the path of the receiver's public key, the path for the
    # output encrypted file, and the sender's private key as parameters.
    with open(public_key_path, 'rb') as file:
    # Opens the file containing the receiver's public key in binary read mode.
        public_key_receiver = RSA.import_key(file.read())
    # Reads the receiver's public key from the file and imports it for use in
    # encryption. This public key will be used to encrypt data so that only the
    # holder of the corresponding private key can decrypt it.

    # Generate a random symmetric key for AES encryption
    symmetric_key = get_random_bytes(32)  # Use a 256-bit key for AES
    iv = get_random_bytes(16)

    # Encrypt the symmetric key with the receiver's public key
    cipher_rsa = PKCS1_OAEP.new(public_key_receiver, hashAlgo=SHA256)
    encrypted_symmetric_key = cipher_rsa.encrypt(symmetric_key)

    # Sign the symmetric key with the sender's private key
    signature = sign_message(symmetric_key, private_key_sender)

    # Encrypt the file content with AES using the symmetric key
    cipher_aes = AES.new(symmetric_key, AES.MODE_CBC, iv=iv)
    with open(file_path, 'rb') as file:
        plaintext = file.read()
    print("Encryption successful.")
    ciphertext = cipher_aes.encrypt(pad(plaintext, AES.block_size))

    # Save the encrypted data, IV, and encrypted signature
    with open(output_path, 'wb') as file:
        file.write(encrypted_symmetric_key)
        file.write(signature)
        file.write(iv)
        file.write(ciphertext)

def decrypt_file(encrypted_path, private_key_path, public_key_sender_path, output_path):
  # Defines a function for decrypting a file. It takes the path of the encrypted file,
  # the path to the receiver's private key, the path to the sender's public key, and the path for the output decrypted file as parameters.
    with open(private_key_path, 'rb') as file:
  # Opens the file containing the receiver's private key in binary read mode.
  # This key is essential for decrypting the data encrypted with the corresponding public key.
        private_key_receiver = RSA.import_key(file.read())
  # Reads and imports the receiver's private key from the file.
  # This key is used to decrypt the encrypted data that was intended for the receiver.

    with open(public_key_sender_path, 'rb') as file:
    # pens the file containing the sender's public key in binary read mode.
    # This public key is used to verify the authenticity of the encrypted data.
        public_key_sender = RSA.import_key(file.read())
    # Reads and imports the sender's public key from the file. This key is necessary
    # to verify any digital signature attached to the encrypted data, ensuring it's from the legitimate sender.


    # Read the encrypted symmetric key, encrypted signature, IV, and ciphertext
    with open(encrypted_path, 'rb') as file:
        encrypted_symmetric_key = file.read(256)
        signature = file.read(256)
        iv = file.read(16)
        ciphertext = file.read()

    # Decrypt the symmetric key with the receiver's private key
    cipher_rsa = PKCS1_OAEP.new(private_key_receiver, hashAlgo=SHA256)
    symmetric_key = cipher_rsa.decrypt(encrypted_symmetric_key)

    # Verify the digital signature of the symmetric key with sender's public key
    if verify_signature(symmetric_key, signature, public_key_sender.export_key()):
        print("Signature verification successful.")
        # Decrypt the file content with AES using the decrypted symmetric key and IV
        cipher_aes = AES.new(symmetric_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher_aes.decrypt(ciphertext), AES.block_size)

        # Save the decrypted file data
        with open(output_path, 'wb') as file:
            file.write(decrypted_data)
        print("Decryption successful.")
    else:
        print("Signature verification failed. Aborting decryption.")

# User 1 (Sender)
private_key_user1, public_key_user1 = generate_rsa_keys()
# Calls the generate_rsa_keys function to create an RSA key pair for User 1, assigning the generated private and public keys to respective variables.
with open('private_key_user1.pem', 'wb') as file:
# Opens (or creates if it doesn't exist) a file named 'private_key_user1.pem' in binary write mode to store User 1's private key.
    file.write(private_key_user1)
with open('public_key_user1.pem', 'wb') as file:
#  Opens a file named 'public_key_user1.pem' in binary write mode for storing User 1's public key.
    file.write(public_key_user1)

# User 2 (Receiver)
private_key_user2, public_key_user2 = generate_rsa_keys()
#  Generates an RSA key pair for User 2 using the generate_rsa_keys function, with separate variables for the private and public keys.
with open('private_key_user2.pem', 'wb') as file:
# Creates or opens a file to store User 2's private key in binary write mode.
    file.write(private_key_user2)
with open('public_key_user2.pem', 'wb') as file:
# Opens or creates a file to store User 2's public key.
    file.write(public_key_user2)

# Encrypt PNG file by User 1
encrypt_file('/content/Fighter.webp', 'public_key_user2.pem', 'encrypted_image.bin', private_key_user1)

# Decrypt PNG file by User 2
decrypt_file('encrypted_image.bin', 'private_key_user2.pem', 'public_key_user1.pem', 'decrypted_image.png')

Encryption successful.
Signature verification successful.
Decryption successful.


https://github.com/VrushangBawne/Mastering-File-Encryption-RSA-AES-in-Python.git
